In [ ]:
p_storage_dir = "../../storage"

In [270]:
import pandas as pd
import numpy as np
from pathlib import Path

DA_STORAGE_DIR = Path(f"{p_storage_dir}/dados_abertos")

# Técnicos Administrativos Educacionais

Para ligar os técnicos às suas respectivas unidades é necessário realizar um cruzamento com informações provenientes de outras fontes de dados, neste caso o [Portal da Transparência](https://www.portaldatransparencia.gov.br/download-de-dados/servidores).

In [271]:
servidores_dados_abertos = pd.read_csv(DA_STORAGE_DIR / "transformed/servidores.csv", delimiter=";")

# Dataset de servidores, extraído do portal da transparência:
servidores_transparencia = pd.read_csv(DA_STORAGE_DIR / "extra/servidores_transparencia.csv", delimiter=";", encoding="L1")

In [272]:
# Servidores Portal da Transparência
servidores_transparencia.head()

,Id_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,DESCRICAO_CARGO,CLASSE_CARGO,REFERENCIA_CARGO,PADRAO_CARGO,NIVEL_CARGO,SIGLA_FUNCAO,...,JORNADA_DE_TRABALHO,DATA_INGRESSO_CARGOFUNCAO,DATA_NOMEACAO_CARGOFUNCAO,DATA_INGRESSO_ORGAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,DIPLOMA_INGRESSO_CARGOFUNCAO,DIPLOMA_INGRESSO_ORGAO,DIPLOMA_INGRESSO_SERVICOPUBLICO,UF_EXERCICIO
0,74721795,ABDALLA ANTONIOS KAYED ELIAS,***.890.691-**,014****,PROFESSOR ENS BASICO TECN TECNOLOGICO,D,0,NaN,402,-1,...,DEDICACAO EXCLUSIVA,01/03/2013,NaN,19/04/2013,000000041,17/02/2004,NaN,PORTARIA,PORTARIA,GO
1,85409914,ABILIO DE JESUS CARRASCAL,***.819.641-**,015****,PROFESSOR ENS BASICO TECN TECNOLOGICO,D,0,NaN,202,-1,...,DEDICACAO EXCLUSIVA,20/05/2014,NaN,08/04/2014,679,07/05/2014,NaN,PORTARIA,PORTARIA,GO
2,84513213,ABISHAI LEMES BORGES NETO,***.011.441-**,017****,TEC DE TECNOLOGIA DA INFORMACAO,D,0,303,0,-1,...,40 HORAS SEMANAIS,02/10/2018,NaN,12/09/2018,2196,26/09/2018,NaN,PORTARIA,PORTARIA,GO
3,73227950,ADAUTO ALVES FILHO,***.220.743-**,000****,VIGILANTE,D,0,416,0,-1,...,40 HORAS SEMANAIS,01/03/2005,NaN,29/12/2008,000000S/N,01/02/1988,NaN,LEI,CONTRATO,GO
4,74910442,ADEL FERNANDO DE ALMEIDA VANNY,***.802.130-**,017****,PROFESSOR ENS BASICO TECN TECNOLOGICO,D,0,NaN,403,-1,...,DEDICACAO EXCLUSIVA,01/03/2013,NaN,25/05/2010,406,15/06/2010,NaN,PORTARIA,PORTARIA,GO


In [273]:
# Servidores Dados Abertos
servidores_dados_abertos.head()

,nome,matricula,cargo,data_ingresso,atribuicao,carga_horaria
0,Abadio Antonio dos Santos,270698,doscente,1958-04-01,PROFESSOR ENS BASICO TECN TECNOLOGICO,40 HORAS SEMANAIS
1,Abdalla Antonios Kayed Elias,1444504,doscente,2004-02-17,PROFESSOR ENS BASICO TECN TECNOLOGICO,DEDICACAO EXCLUSIVA
2,Abel Ferreira de Lima,6050871,técnico,1985-03-06,MOTORISTA,40 HORAS SEMANAIS
3,Abel Ferreira de Lima,50871,técnico,1985-03-06,MOTORISTA,40 HORAS SEMANAIS
4,Abelson Oliveira de Andrade,50644,técnico,1969-07-01,MOTORISTA,40 HORAS SEMANAIS


## Portal da Transparência - pré-processamento dos técnicos

1. Separar os TAEs dos demais
2. Separar as colunas importantes
3. Renomear as colunas
4. Formatar as datas para o padrão "yyyy-mm-dd"
5. Troca os valores da coluna "uorg_exercicio" para a sigla da respectiva unidade

In [274]:
# Filtra o dataset e armazena apenas os TAEs em uma variável
taes_transparencia = servidores_transparencia[(servidores_transparencia["TIPO_VINCULO"] == "Cargo") & (servidores_transparencia["DESCRICAO_CARGO"] != "PROFESSOR ENS BASICO TECN TECNOLOGICO") & (servidores_transparencia["DESCRICAO_CARGO"] != "PROF ENS BAS TEC TECNOLOGICO-SUBSTITUTO")]

# Fatia o dataset deixando apenas as colunas que nos importam
taes_transparencia = taes_transparencia[["NOME", "UORG_EXERCICIO", "DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO"]].reset_index(drop=True)

# Renomeia a coluna "DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO" para "DATA_INGRESSO"
taes_transparencia.rename(columns={"DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO": "data_ingresso", "NOME": "nome", "UORG_EXERCICIO": "uorg_exercicio"}, inplace=True)

taes_transparencia.head(n=10)

,nome,uorg_exercicio,data_ingresso
0,ABISHAI LEMES BORGES NETO,OES-COORD. DE TECNOLOGIA DA INFORMACAO,26/09/2018
1,ADAUTO ALVES FILHO,GYN-COORDENACAO DE BIBLIOTECA,01/02/1988
2,ADEMAR LOURENCO ALVES JUNIOR,GYN-COORD. DE APOIO ADM. - AREA 4,25/07/2016
3,ADILSON CORREIA GOULART,ITU-COORDENACAO DE APOIO ADMINISTRATIVO,19/04/2012
4,ADOLFO MARTINS DA SILVA,APA-COORDENACAO DE APOIO ADMINISTRATIVO,19/04/2012
5,ADRIAM MARCOS DA SILVA,OES-GERENCIA DE ADMINISTRACAO,05/08/2016
6,ADRIANA DE ASSIS DAMASCENO,ITU-COORD. DE INTERACAO ESCOLA-EMPRESA,01/09/2008
7,ADRIANA RAQUEL SANTOS MEDEIROS,REI-SEGURANCA DO TRABALHO SIASS,18/08/2014
8,ADRIANA ROSELY SILVA,LIN-COORDENACAO DE APOIO ADMINISTRATIVO,01/08/2005
9,ADRIANA SOUZA CAMPOS,REI-DIRETORIA DE COMUNICACAO SOCIAL,06/01/2012


In [275]:
# Converte a data do formato BR (dd/mm/yyyy) para o formato ISO (yyyy/mm/dd)
def to_iso_date(br_date) -> str:
    if br_date is np.nan:
        return ""
    
    sliced = br_date.split("/")
    return f"{sliced[2]}-{sliced[1]}-{sliced[0]}"

datas_transformadas = []

for data in taes_transparencia["data_ingresso"]:
    datas_transformadas.append(to_iso_date(data))

In [276]:
# Substitui os valores da coluna "data_ingresso"
taes_transparencia["data_ingresso"] = datas_transformadas

taes_transparencia.head(n=10)

,nome,uorg_exercicio,data_ingresso
0,ABISHAI LEMES BORGES NETO,OES-COORD. DE TECNOLOGIA DA INFORMACAO,2018-09-26
1,ADAUTO ALVES FILHO,GYN-COORDENACAO DE BIBLIOTECA,1988-02-01
2,ADEMAR LOURENCO ALVES JUNIOR,GYN-COORD. DE APOIO ADM. - AREA 4,2016-07-25
3,ADILSON CORREIA GOULART,ITU-COORDENACAO DE APOIO ADMINISTRATIVO,2012-04-19
4,ADOLFO MARTINS DA SILVA,APA-COORDENACAO DE APOIO ADMINISTRATIVO,2012-04-19
5,ADRIAM MARCOS DA SILVA,OES-GERENCIA DE ADMINISTRACAO,2016-08-05
6,ADRIANA DE ASSIS DAMASCENO,ITU-COORD. DE INTERACAO ESCOLA-EMPRESA,2008-09-01
7,ADRIANA RAQUEL SANTOS MEDEIROS,REI-SEGURANCA DO TRABALHO SIASS,2014-08-18
8,ADRIANA ROSELY SILVA,LIN-COORDENACAO DE APOIO ADMINISTRATIVO,2005-08-01
9,ADRIANA SOUZA CAMPOS,REI-DIRETORIA DE COMUNICACAO SOCIAL,2012-01-06


In [277]:
# Substitui a identificação completa da uorg apenas pela sigla
siglas_uorg = [uorg[:3] for uorg in taes_transparencia["uorg_exercicio"]]

taes_transparencia["uorg_exercicio"] = siglas_uorg

# Adiciona uma coluna com os nomes em lowercase para fazer o merge
taes_transparencia["nome_lower"] = [n.lower() for n in taes_transparencia["nome"]]

In [278]:
taes_transparencia.head(n=10)

,nome,uorg_exercicio,data_ingresso,nome_lower
0,ABISHAI LEMES BORGES NETO,OES,2018-09-26,abishai lemes borges neto
1,ADAUTO ALVES FILHO,GYN,1988-02-01,adauto alves filho
2,ADEMAR LOURENCO ALVES JUNIOR,GYN,2016-07-25,ademar lourenco alves junior
3,ADILSON CORREIA GOULART,ITU,2012-04-19,adilson correia goulart
4,ADOLFO MARTINS DA SILVA,APA,2012-04-19,adolfo martins da silva
5,ADRIAM MARCOS DA SILVA,OES,2016-08-05,adriam marcos da silva
6,ADRIANA DE ASSIS DAMASCENO,ITU,2008-09-01,adriana de assis damasceno
7,ADRIANA RAQUEL SANTOS MEDEIROS,REI,2014-08-18,adriana raquel santos medeiros
8,ADRIANA ROSELY SILVA,LIN,2005-08-01,adriana rosely silva
9,ADRIANA SOUZA CAMPOS,REI,2012-01-06,adriana souza campos


In [279]:
taes_transparencia.shape

(892, 4)

## Portal de Dados Abertos - pré-processamento dos técnicos

1. Filtrar apenas os técnicos
2. Remover a coluna 'cargo'

In [280]:
# Filtra apenas os técnicos
taes_dados_abertos = servidores_dados_abertos[servidores_dados_abertos.cargo == "técnico"]

# Remove a coluna cargo
taes_dados_abertos = taes_dados_abertos.drop("cargo", axis=1)

taes_dados_abertos.head(n=10)

# Adiciona uma coluna com os nomes em lowercase para fazer o merge
taes_dados_abertos["nome_lower"] = [n.lower() for n in taes_dados_abertos["nome"]]

In [281]:
taes_dados_abertos.head()

,nome,matricula,data_ingresso,atribuicao,carga_horaria,nome_lower
2,Abel Ferreira de Lima,6050871,1985-03-06,MOTORISTA,40 HORAS SEMANAIS,abel ferreira de lima
3,Abel Ferreira de Lima,50871,1985-03-06,MOTORISTA,40 HORAS SEMANAIS,abel ferreira de lima
4,Abelson Oliveira de Andrade,50644,1969-07-01,MOTORISTA,40 HORAS SEMANAIS,abelson oliveira de andrade
6,Abigail Rodrigues Silva,271086,1990-02-01,AUXILIAR DE ENFERMAGEM,40 HORAS SEMANAIS,abigail rodrigues silva
10,Abishai Lemes Borges Neto,1744883,2018-09-26,TEC DE TECNOLOGIA DA INFORMACAO,40 HORAS SEMANAIS,abishai lemes borges neto


## Merge dos dois datasets

In [282]:
taes_merged = taes_dados_abertos.merge(taes_transparencia.iloc[:, 1:], on=["nome_lower", "data_ingresso"]).drop(columns="nome_lower")

In [283]:
taes_merged.head(n=10)

,nome,matricula,data_ingresso,atribuicao,carga_horaria,uorg_exercicio
0,Abishai Lemes Borges Neto,1744883,2018-09-26,TEC DE TECNOLOGIA DA INFORMACAO,40 HORAS SEMANAIS,OES
1,Adauto Alves Filho,47364,1988-02-01,VIGILANTE,40 HORAS SEMANAIS,GYN
2,Ademar Lourenco Alves Junior,2326904,2016-07-25,TECNICO DE LABORATORIO AREA,40 HORAS SEMANAIS,GYN
3,Adilson Correia Goulart,1938714,2012-04-19,AUX EM ADMINISTRACAO,40 HORAS SEMANAIS,ITU
4,Adolfo Martins da Silva,1941628,2012-04-19,TECNICO EM AUDIOVISUAL,40 HORAS SEMANAIS,APA
5,Adriam Marcos da Silva,2330091,2016-08-05,ASSISTENTE DE LABORATORIO,40 HORAS SEMANAIS,OES
6,Adriana de Assis Damasceno,2654432,2008-09-01,TECNICO EM ASSUNTOS EDUCACIONAIS,40 HORAS SEMANAIS,ITU
7,Adriana de Assis Damasceno,1654432,2008-09-01,ASSISTENTE EM ADMINISTRACAO,40 HORAS SEMANAIS,ITU
8,Adriana Raquel Santos Medeiros,2153955,2014-08-18,TEC EM SEGURANCA DO TRABALHO,40 HORAS SEMANAIS,REI
9,Adriana Rosely Silva,1880832,2005-08-01,TECNICO EM ASSUNTOS EDUCACIONAIS,40 HORAS SEMANAIS,LIN


## Exportação do dataset gerado

In [284]:
export_path = Path(DA_STORAGE_DIR / "preprocessed")

if not export_path.is_dir():
    Path(export_path).mkdir()

try:
    taes_merged.to_csv(export_path / "taes.csv", sep=";", index=False)
except Exception as e:
    print(str(e))